In [ ]:
import glob
import json
import os

In [ ]:
ls data

In [ ]:
scout_data_dir = 'data'


In [ ]:
result_files = glob.glob('data/scoutsuite_results*.js')
result_files

In [ ]:
def load_results(scout_data_dir):
    scout_data = {}
    result_files = glob.glob('data/scoutsuite_results*.js')
    for result_file in result_files:
        account_alias = result_file.split('.')[0].split('_')[-1].split('-',1)[1]
        with open(result_file, 'r') as f:
            # remove the first line 'scout_results =' of .js file
            data = [line.rstrip('\n') for line in f.readlines()[1:]]
            scout_data[account_alias] = json.loads("".join(data))
    return scout_data

scout_data = load_results(scout_data_dir)
print(scout_data.keys())
print([scout_data[alias]['account_id'] for alias in scout_data.keys()])


In [ ]:
first_account = scout_data[list(scout_data.keys())[0]]
first_account['service_list']

In [ ]:
first_account.keys()

In [ ]:
first_account['account_id']

In [ ]:
first_account['metadata']

In [ ]:
first_account['services']['iam'].keys()

In [ ]:
first_account['services']['iam']['findings']

In [ ]:
service = first_account['services']['iam']
for finding_name, finding_data in service['findings'].items():
    print(finding_name)
    for item in finding_data['items'][0:2]:
        svc,obj,name,*rest = item.split('.')
        print(name)
        break

In [ ]:
first_account['services']['iam']['findings']['iam-assume-role-lacks-external-id-and-mfa']

In [ ]:
%pdb
from collections import defaultdict
import pandas as pd

def print_findings(scout_data, services=[], account_aliases=[], output='csv'):
    '''
    Print findings across all accounts with opinionated formatting.
    @scout_data: dict with keys of account_aliases
    @account_aliases: account_alias matching --profile in ScoutSuite run. If present, only print this data
    @output: None -> print to stdout, csv -> comma separated, xls -> excel
    '''
    # finding_data is the same data, but key-walk inverted so the leafs are the stems of the data path
    # using solution that allows mkdir -p behavior for d['path']['to']['data']
    # https://stackoverflow.com/questions/60808884/python-to-create-dict-keys-path-similarly-to-mkdir-p
    frame = pd.DataFrame(columns=['Account', 'Finding', 'Resource'])
    findings = {} #defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

    service_display_map = {'iam': 3, 'cloudtrail': 5}
    for alias, data in scout_data.items():
        account_id = data['account_id']
        if account_aliases and alias not in account_aliases:
            continue
        for service_name, service_data in data['services'].items():
            if services and service_name not in services:
                continue
            for finding_name, finding_data in service_data['findings'].items():
                if 'snapshot' in finding_name:
                    continue
                if finding_name not in findings:
                    findings[finding_name] = {}
                if finding_name in findings and not alias in findings[finding_name]:
                    findings[finding_name][alias] = {}
                checked_items = finding_data['checked_items']
                flagged_items = []
                #print("finding_data.keys()", finding_data.keys())
                if 'display_path' in finding_data:
                    display_path_depth = len(finding_data['display_path'].split('.'))
                elif 'path' in finding_data:
                    display_path_depth = len(finding_data['path'].split('.'))
                else:
                    display_path_depth = service_display_map.get(service_name, 0)
                for item in finding_data['items']:
                    # eg. iam, roles, AROA..., assume_role_policy, PolicyDocument, Statement, 0
                    parts = item.split('.')
                    display_value = ''
                    try:
                        def path_get(data, path_parts):
                            temp = data
                            for part in path_parts:
                                if part not in temp:
                                    print(f'ERROR: path element {part} not in {temp}')
                                    print('finding_data', finding_data)
                                    print(f'no display_path in {finding_name} setting display_path_depth to {display_path_depth}\n')
                                    return ""
                                temp = temp[part]
                            return temp['name']

                        display_value = path_get(service_data, parts[1:display_path_depth])
                        flagged_items.append(display_value)
                        frame.append([account_id, finding_name, display_value])
                    except Exception as e:
                        print(e)
                        print(finding_name)
                        print(item)
                        print('parts', parts)
                        print('display_path_depth', display_path_depth)
                        print(f"{display_value} not in {service_data[parts[1]].keys()}".format())
                        break
                    # regions duplicate some findings, deduplicate with list(set())
                    findings[finding_name][alias]['flagged_items'] = list(set(flagged_items))
                    findings[finding_name][alias]['checked_items'] = checked_items
           
    print("\n***********************\n")
    for finding_name, finding_data in findings.items():
        print(finding_name, '\n')
        for alias, account_data in finding_data.items():
            flagged_items = account_data.get('flagged_items', [])
            checked_items = account_data.get('checked_items', 0)
            if len(flagged_items) > 0:
                print(f"**{alias}**", "{} of {} failed\n".format(len(flagged_items), checked_items))
                print(" \\\n".join(flagged_items))
                print("\n")
    return frame

In [ ]:
groupings = {
    'aws_security_groups_allow_all_traffic_to_sensitive_ports': {
        'ec2': ['ec2-security-group-opens-DNS-port-to-all', 
                'ec2-security-group-opens-MongoDB-port-to-all', 
                'ec2-security-group-opens-MsSQL-port-to-all', 
                'ec2-security-group-opens-MySQL-port-to-all', 
                'ec2-security-group-opens-NFS-port-to-all', 
                'ec2-security-group-opens-Oracle DB-port-to-all', 
                'ec2-security-group-opens-PostgreSQL-port-to-all', 
                'ec2-security-group-opens-RDP-port-to-all', 
                'ec2-security-group-opens-SMTP-port-to-all', 
                'ec2-security-group-opens-SSH-port-to-all', 
                'ec2-security-group-opens-TCP-port-to-all', 
                'ec2-security-group-opens-UDP-port-to-all', 
                'ec2-security-group-opens-all-ports', 
                'ec2-security-group-opens-all-ports-to-all', 
                'ec2-security-group-opens-all-ports-to-self', 
                'ec2-security-group-opens-icmp-to-all', 
                'ec2-security-group-opens-plaintext-port-FTP', 
                'ec2-security-group-opens-plaintext-port-Telnet']},
    'aws_elb_traffic_encryption_not_enforced': {
        'elbv2': ['elb-listener-allowing-cleartext'],
        'elb': ['elbv2-listener-allowing-cleartext']}
}

Once we have ironed out all the bugs, we can simply apply the desired groupings mapped to VKB templates and auto
populate the finding. For now, we print findings service by service.

In [ ]:
print_findings(scout_data, services = ['ec2'])

In [ ]:
print_findings(scout_data, services = ['iam']) #, account_aliases=['DF01','DF13'])

In [ ]:
print_findings(scout_data, services = ['cloudtrail'])

In [ ]:
print_findings(scout_data, services = ['elb'])

In [ ]:
print_findings(scout_data, services = ['elbv2'])

In [ ]:
print_findings(scout_data, services = ['ec2'])

In [ ]:
scout_data['DF01']['services']['s3']['findings'].keys()

In [ ]:
print_findings(scout_data, services = ['s3'])

In [ ]:
scout_data['DF01']['services']['ec2']['findings'].keys()

In [ ]:
scout_data['DF01']['services']['ec2']['findings']['ec2-security-group-whitelists-aws']

In [ ]:
scout_data['DF01']['services']['elb']['findings']['elb-listener-allowing-cleartext']

In [ ]:
scout_data['DF01']['services']['ec2']['findings']['ec2-instance-with-user-data-secrets']

In [ ]:
scout_data['DF01']['services']['iam']['findings']['iam-inline-role-policy-allows-sts-AssumeRole']

In [ ]:
service.keys()

In [ ]:
service['findings']['iam-assume-role-lacks-external-id-and-mfa'].keys()

In [ ]:
item = service['findings']['password-policy-expiration-threshold']['items'][0]
item

In [ ]:
svc,service_entity,name,*rest = item.split('.')
name

In [ ]:
service['roles'][name]

In [ ]:
scout_data.keys()

In [ ]:
scout_data['DF13']['services']['cloudtrail'].keys()

In [ ]:
scout_data['DF13']['services']['cloudtrail']['regions']['us-east-1']['trails']

In [ ]:
scout_data['DF13']['services']['cloudtrail']['findings']['cloudtrail-no-encryption-with-kms']

In [ ]:
first_account['services']['cloudtrail']['findings']['cloudtrail-no-encryption-with-kms']

In [ ]:
def path_get(data, path_parts):
    temp = data
    for part in path_parts:
        temp = temp[part]
    return temp

In [ ]:
path_get(scout_data, ['DF13','services','iam', 'findings', 'iam-assume-role-lacks-external-id-and-mfa'])

In [ ]:
scout_data['DF13']['services']['iam']['findings']['iam-assume-role-lacks-external-id-and-mfa'].keys()

In [ ]:
svc,service_entity,name,*rest = item.split('.')
                    try:
                        flagged_items.append(service_data[service_entity][name]['arn'])

**Nmap filtering**


In [ ]:
nmap = nmap.replace('\\', '')
for line in nmap.split('\n'):
    if line.startswith('**'):
        account = line #.strip('**')
        print('\n')
        print(account, '\n')
    elif line.startswith('Host:'):
        host = line.lstrip('Host: ').rstrip('\\').strip()
        print(host)
    elif line.startswith('Ports: '):
        ports = line.lstrip('Ports: ').strip('\\').strip().split(',')
        open_ports = [port.lstrip('Ports:').replace('/open', '').rstrip('/').strip() for port in ports if 'open' in port]
        print(open_ports)
        

In [ ]:
nmap